**Set environment**

In [2]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Prepare

**Set global variable**

In [3]:
TXT_FNAME_INP = "region.annotation.encode_e2g_benchmark.tsv"
TXT_FNAME_OUT = "region.annotation.genome_tss.tsv"

**View files**

In [4]:
### get annotated region folder
txt_fdiry = file.path(FD_RES, "region_annotation")
vec = dir(txt_fdiry)

### assign and show
vec_txt_folder = vec
for(txt in vec){cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


In [5]:
txt_fdiry = file.path(FD_RES, "region_annotation", "*", "summary")
txt_fname = TXT_FNAME_INP
txt_fglob = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for(txt in vec){cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_overlap/summary/region.annotation.encode_e2g_benchmark.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_union/summary/region.annotation.encode_e2g_benchmark.tsv 


## Import data

**Import region pairs**

In [6]:
### loop to import data
lst = lapply(vec_txt_folder, function(txt_folder){
    ### set file directory
    txt_fdiry = file.path(FD_RES, "region_annotation", txt_folder, "summary")
    txt_fname = TXT_FNAME_INP
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### read table
    dat = read_tsv(txt_fpath, show_col_types = FALSE)
    return(dat)
})
names(lst) = vec_txt_folder

### assign and show
lst_dat_region_annot_import = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
fun_display_table(head(dat, 3))

$fcc_astarr_macs_input_overlap
[1] 8770   11

$fcc_astarr_macs_input_union
[1] 9229   11



Chrom,ChromStart,ChromEnd,Region,Group,Label,Target,Score,NLog10P,Regulated,Source
chr1,3774056,3776283,chr1:3774056-3776283,E2G-Benchmark,Regulated:TRUE,CEP104,-0.2934319,2.395344,TRUE,Ulirsch2016
chr1,3774056,3776283,chr1:3774056-3776283,E2G-Benchmark,Regulated:TRUE,LRRC47,-0.3311781,2.109514,TRUE,Ulirsch2016
chr1,3774056,3776283,chr1:3774056-3776283,E2G-Benchmark,Regulated:TRUE,SMIM1,-0.4720192,3.192703,TRUE,Ulirsch2016


## Summarize

In [8]:
### rename columns
lst = lst_dat_region_annot_import
lst = lapply(lst, function(dat){
    ###
    dat1 = dat %>%
        dplyr::group_by(Chrom, ChromStart, ChromEnd, Region) %>%
        dplyr::summarize(
            Regulated_Ifany  = any(Regulated),
            Regulated_Count  = sum(Regulated),
            .groups = "drop"
        )
    ###
    dat2 = dat %>%
        dplyr::filter(Regulated) %>%
        dplyr::group_by(Chrom, ChromStart, ChromEnd, Region) %>%
        dplyr::summarize(
            Regulated_Target = paste(unique(Target), collapse = ","),
            .groups = "drop"
        )
    ###
    dat3 = dplyr::left_join(
        dat1, dat2, 
        by = c("Chrom", "ChromStart", "ChromEnd", "Region")
    )
    return(dat3)
})

### assign and show
lst_dat_region_annot_result = lst
cat("Region:", names(lst), "\n\n")
dat = lst[[1]]
fun_display_table(head(dat, 3))

Region: fcc_astarr_macs_input_overlap fcc_astarr_macs_input_union 



Chrom,ChromStart,ChromEnd,Region,Regulated_Ifany,Regulated_Count,Regulated_Target
chr1,3774056,3776283,chr1:3774056-3776283,TRUE,3,"CEP104,LRRC47,SMIM1"
chr1,3803955,3806146,chr1:3803955-3806146,FALSE,0,NA
chr1,4126841,4128109,chr1:4126841-4128109,FALSE,0,NA


## Export

In [10]:
lst = lst_dat_region_annot_result
for (idx in names(lst)){

    ### get
    dat_region_annot_result = lst_dat_region_annot_result[[idx]]
    
    ### set file directory
    txt_folder = idx
    txt_fdiry = file.path(
        FD_RES, 
        "region_annotation", 
        txt_folder,
        "summary"
    )
    
    ###
    txt_fname = "region.annotation.encode_e2g_benchmark.label.tsv"
    txt_fname = "region.summary.encode_e2g_benchmark.label.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)
    
    dat = dat_region_annot_result
    dat = dat %>% dplyr::arrange(Chrom, ChromStart, ChromEnd)
    #write_tsv(dat, txt_fpath)
}